<a href="https://colab.research.google.com/github/Blecyrajan/ML_Projects/blob/main/Copy_of_Brain_Tumor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Environment & Kaggle Data Setup**

In [ ]:
import kagglehub
path = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

**Data Preprocessing & Augmentation**

In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


IMAGE_SIZE = 224
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('/kaggle/input/brain-tumor-mri-dataset/Training', transform=transform)
test_dataset = datasets.ImageFolder('/kaggle/input/brain-tumor-mri-dataset/Testing', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

classes = train_dataset.classes
print(f"Classes: {classes}")


**The Hybrid CNN-ViT Architecture**

In [ ]:
import torch.nn as nn
import timm

class BrainTumorModel(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        # CNN: ResNet18 for local texture
        self.cnn = timm.create_model('resnet18', pretrained=True, features_only=True)
        # ViT: Tiny Vision Transformer for global context
        self.vit = timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=0)

        # Fusion Layer
        self.classifier = nn.Linear(512 + 192, num_classes)

    def forward(self, x):
        # 1. Get CNN Features
        cnn_maps = self.cnn(x)[-1] # [B, 512, 7, 7]
        cnn_feat = torch.mean(cnn_maps, dim=(2, 3)) # Global Average Pool

        # 2. Get ViT Features
        vit_feat = self.vit(x) # [B, 192]

        # 3. Combine
        combined = torch.cat((cnn_feat, vit_feat), dim=1)
        logits = self.classifier(combined)

        return logits, cnn_maps # Maps used for RSC Loss & Intrinsic Heatmaps


**RSC Loss & Training Logic**

In [ ]:
def rsc_loss_fn(feat_orig, feat_flipped):
    # Intrinsic Explanation Map Generation
    map_orig = torch.mean(feat_orig, dim=1)
    map_flipped = torch.mean(feat_flipped, dim=1)
    map_flipped_restored = torch.flip(map_flipped, dims=[2])
    return nn.functional.mse_loss(map_orig, map_flipped_restored)

# Training Settings
device = "cuda" if torch.cuda.is_available() else "cpu"
model = BrainTumorModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) # AdamW is better for ViT
criterion = nn.CrossEntropyLoss()

# Training Loop
def train_model(epochs=10):
    for epoch in range(epochs):
        model.train()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward Original
            logits, feat_orig = model(images)
            # Forward Flipped (for Consistency)
            _, feat_flipped = model(torch.flip(images, dims=[-1])) # Assuming images are [B, C, H, W], flip along W

            # Weighted Loss
            loss_cls = criterion(logits, labels)
            loss_rsc = rsc_loss_fn(feat_orig, feat_flipped)
            total_loss = loss_cls + (0.5 * loss_rsc)

            total_loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            train_loss += total_loss.item()

        print(f"Epoch {epoch+1} | Loss: {train_loss/len(train_loader):.4f}")

train_model(epochs=10) # Increase epochs for better accuracy

**Scientific Evaluation & Plotting**

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import cv2

def evaluate_and_save_plots():
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out, _ = model(imgs)
            preds.extend(out.argmax(1).cpu().numpy())
            targets.extend(labels.cpu().numpy())

    # 1. Confusion Matrix Plot
    plt.figure(figsize=(10,8))
    cm = confusion_matrix(targets, preds)
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix: CNN-ViT RSC Model')
    plt.savefig('confusion_matrix.png')
    plt.show()

    # 2. Intrinsic Heatmap Plot
    sample_img, _ = test_dataset[10] # Pick a sample
    logits, maps = model(sample_img.unsqueeze(0).to(device))

    # Process heatmap from the internal learned layer
    heatmap = torch.mean(maps, dim=1).squeeze().cpu().detach().numpy()
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())
    heatmap = cv2.resize(heatmap, (224, 224))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(sample_img.permute(1, 2, 0).cpu())
    plt.title("Original MRI")

    plt.subplot(1, 2, 2)
    plt.imshow(heatmap, cmap='jet')
    plt.title("Intrinsic RSC Explanation")
    plt.savefig("explanation_results.png")
    plt.show()

evaluate_and_save_plots()

**Saving for Production**

In [ ]:
# Save the entire model (weights and architecture)
torch.save(model.state_dict(), 'brain_tumor_rsc_model.pth')

# To load in Backend:
# model = BrainTumorModel()
# model.load_state_dict(torch.load('brain_tumor_rsc_model.pth'))
# model.eval()


**Dataset Distribution & Sample Visualization**

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch

# --- Plot A: Dataset Distribution ---
def plot_dataset_summary(dataset):
    targets = np.array(dataset.targets)
    class_counts = [np.sum(targets == i) for i in range(len(dataset.classes))]

    plt.figure(figsize=(10, 6), dpi=100)
    sns.barplot(x=dataset.classes, y=class_counts, palette="magma")
    plt.title("Distribution of Brain Tumor Classes in Dataset", fontsize=14)
    plt.ylabel("Number of Images")
    plt.savefig("figure_1_distribution.png", bbox_inches='tight')
    plt.show()

# --- Plot B: Samples from Each Class ---
def plot_class_samples(dataset):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for i, class_name in enumerate(dataset.classes):
        # Find first index of this class
        idx = dataset.targets.index(i)
        img, _ = dataset[idx]
        img = img.permute(1, 2, 0).numpy()
        # Un-normalize for display
        img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]
        axes[i].imshow(np.clip(img, 0, 1))
        axes[i].set_title(f"Class: {class_name}", fontsize=12)
        axes[i].axis('off')
    plt.savefig("figure_2_samples.png")
    plt.show()

plot_dataset_summary(train_dataset)
plot_class_samples(train_dataset)


**Preprocessing Analysis (Before & After)**

In [ ]:
def plot_preprocessing_effect(dataset_path):
    from PIL import Image
    raw_img = Image.open(train_dataset.imgs[0][0]) # Load one raw file

    # Process it using  transform
    processed_img, _ = train_dataset[0]
    processed_img = processed_img.permute(1, 2, 0).numpy()
    processed_img = (processed_img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(raw_img)
    plt.title("Before: Raw MRI (Varying Size/Intensity)")

    plt.subplot(1, 2, 2)
    plt.imshow(np.clip(processed_img, 0, 1))
    plt.title("After: Preprocessed (224x224 & Normalized)")
    plt.savefig("figure_3_preprocessing.png")
    plt.show()

plot_preprocessing_effect('brain_data/Training')


**Training History (Loss & Accuracy)**

In [ ]:
def plot_training_history(history):
    epochs = range(1, len(history['train_loss']) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss Curve
    ax1.plot(epochs, history['train_loss'], 'b-', label='Training Loss')
    ax1.plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
    ax1.set_title('Training vs. Validation Loss')
    ax1.legend()

    # Accuracy Curve
    ax2.plot(epochs, history['train_acc'], 'b-', label='Training Acc')
    ax2.plot(epochs, history['val_acc'], 'r-', label='Validation Acc')
    ax2.set_title('Training vs. Validation Accuracy')
    ax2.legend()

    plt.savefig("figure_4_history.png")
    plt.show()


 **Advanced Evaluation (Confusion Matrix & ROC)**

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc

def plot_advanced_metrics(model, loader, classes):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs, _ = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.numpy())

    # --- Confusion Matrix ---
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title("Confusion Matrix")
    plt.savefig("figure_5_cm.png")

    # --- ROC / AUC ---
    plt.figure(figsize=(10, 8))
    all_probs = np.array(all_probs)
    for i in range(len(classes)):
        fpr, tpr, _ = roc_curve(np.array(all_labels) == i, all_probs[:, i])
        plt.plot(fpr, tpr, label=f'{classes[i]} (AUC = {auc(fpr, tpr):.3f})')
    plt.plot(,, 'k--')
    plt.title("Multiclass ROC Curves")
    plt.legend()
    plt.savefig("figure_6_roc.png")

plot_advanced_metrics(model, test_loader, classes)


In [ ]:
def rsc_loss_fn(feat_orig, feat_flipped):
    # Intrinsic Explanation Map Generation
    map_orig = torch.mean(feat_orig, dim=1)
    map_flipped = torch.mean(feat_flipped, dim=1)
    map_flipped_restored = torch.flip(map_flipped, dims=[2])
    return nn.functional.mse_loss(map_orig, map_flipped_restored)

# Training Settings
device = "cuda" if torch.cuda.is_available() else "cpu"
model = BrainTumorModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) # AdamW is better for ViT
criterion = nn.CrossEntropyLoss()

# Training Loop
def train_model(epochs=10, train_loader=train_loader, test_loader=test_loader):
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward Original
            logits, feat_orig = model(images)
            # Forward Flipped (for Consistency)
            _, feat_flipped = model(torch.flip(images, dims=[-1])) # Assuming images are [B, C, H, W], flip along W

            # Weighted Loss
            loss_cls = criterion(logits, labels)
            loss_rsc = rsc_loss_fn(feat_orig, feat_flipped)
            total_loss = loss_cls + (0.5 * loss_rsc)

            total_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            running_train_loss += total_loss.item()
            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = running_train_loss / len(train_loader)
        train_accuracy = 100 * correct_train / total_train

        # Validation phase
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                logits, _ = model(images)
                loss = criterion(logits, labels)

                running_val_loss += loss.item()
                _, predicted = torch.max(logits.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        avg_val_loss = running_val_loss / len(test_loader)
        val_accuracy = 100 * correct_val / total_val

        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    return history

history = train_model(epochs=10) # Increase epochs for better accuracy

In [ ]:
def plot_training_history(history):
    epochs = range(1, len(history['train_loss']) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss Curve
    ax1.plot(epochs, history['train_loss'], 'b-', label='Training Loss')
    ax1.plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
    ax1.set_title('Training vs. Validation Loss')
    ax1.legend()

    # Accuracy Curve
    ax2.plot(epochs, history['train_acc'], 'b-', label='Training Acc')
    ax2.plot(epochs, history['val_acc'], 'r-', label='Validation Acc')
    ax2.set_title('Training vs. Validation Accuracy')
    ax2.legend()

    plt.savefig("figure_4_history.png")
    plt.show()

plot_training_history(history)

In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report

def plot_advanced_metrics(model, loader, classes):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs, _ = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.numpy())

    # --- Classification Report ---
    print("\nClassification Report:\n")
    print(classification_report(all_labels, all_preds, target_names=classes))

    # --- Confusion Matrix ---
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title("Confusion Matrix")
    plt.savefig("figure_5_cm.png")
    plt.show()

    # --- ROC / AUC ---
    plt.figure(figsize=(10, 8))
    all_probs = np.array(all_probs)
    for i in range(len(classes)):
        fpr, tpr, _ = roc_curve(np.array(all_labels) == i, all_probs[:, i])
        plt.plot(fpr, tpr, label=f'{classes[i]} (AUC = {auc(fpr, tpr):.3f})')
    plt.plot([0, 1], [0, 1], 'k--') # Fixed: added [0,1], [0,1]
    plt.title("Multiclass ROC Curves")
    plt.legend()
    plt.savefig("figure_6_roc.png")
    plt.show()

plot_advanced_metrics(model, test_loader, classes)

# Task
The final goal is to develop a robust brain tumor MRI classification system using a hybrid CNN-ViT architecture and Radiological Symmetry Loss (RSC). This involves downloading and setting up the "masoudnickparvar/brain-tumor-mri-dataset", visualizing dataset characteristics (class distribution and sample images), defining preprocessing steps including resizing and normalization, and illustrating their effect. The core of the task is defining and training a `BrainTumorModel` that combines ResNet18 for local features and ViT Tiny for global context, along with a custom `rsc_loss_fn` to enhance interpretability. The model will be trained over 10 epochs, with training history (loss and accuracy) visualized to monitor performance. Finally, a comprehensive evaluation will be conducted on the test set, including a classification report, confusion matrix, multi-class ROC curves with AUC scores, and intrinsic heatmaps for explainability, before saving the trained model to 'brain_tumor_rsc_model.pth'.

## Dataset Characteristics Visualization

### Subtask:
Plot the distribution of classes within the training dataset, showing the number of images for each tumor type. Additionally, visualize sample images from each class to provide a visual understanding of the dataset's content.


**Reasoning**:
The user explicitly asked to run the code in the cell that defines the `plot_dataset_summary` and `plot_class_samples` functions. This will generate and display the required visualizations for dataset characteristics.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch

# --- Plot A: Dataset Distribution ---
def plot_dataset_summary(dataset):
    targets = np.array(dataset.targets)
    class_counts = [np.sum(targets == i) for i in range(len(dataset.classes))]

    plt.figure(figsize=(10, 6), dpi=100)
    sns.barplot(x=dataset.classes, y=class_counts, palette="magma")
    plt.title("Distribution of Brain Tumor Classes in Dataset", fontsize=14)
    plt.ylabel("Number of Images")
    plt.savefig("figure_1_distribution.png", bbox_inches='tight')
    plt.show()

# --- Plot B: Samples from Each Class ---
def plot_class_samples(dataset):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for i, class_name in enumerate(dataset.classes):
        # Find first index of this class
        idx = dataset.targets.index(i)
        img, _ = dataset[idx]
        img = img.permute(1, 2, 0).numpy()
        # Un-normalize for display
        img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]
        axes[i].imshow(np.clip(img, 0, 1))
        axes[i].set_title(f"Class: {class_name}", fontsize=12)
        axes[i].axis('off')
    plt.savefig("figure_2_samples.png")
    plt.show()

plot_dataset_summary(train_dataset)
plot_class_samples(train_dataset)

**Reasoning**:
The previous code failed because `train_dataset` was not defined in the scope of the plotting functions. I will include the necessary data loading and transformation code from earlier cells to ensure `train_dataset` is properly initialized before being used by the plotting functions.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Re-initialize dataset and transform for this block
IMAGE_SIZE = 224
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder('/kaggle/input/brain-tumor-mri-dataset/Training', transform=transform)

# --- Plot A: Dataset Distribution ---
def plot_dataset_summary(dataset):
    targets = np.array(dataset.targets)
    class_counts = [np.sum(targets == i) for i in range(len(dataset.classes))]

    plt.figure(figsize=(10, 6), dpi=100)
    sns.barplot(x=dataset.classes, y=class_counts, palette="magma")
    plt.title("Distribution of Brain Tumor Classes in Dataset", fontsize=14)
    plt.ylabel("Number of Images")
    plt.savefig("figure_1_distribution.png", bbox_inches='tight')
    plt.show()

# --- Plot B: Samples from Each Class ---
def plot_class_samples(dataset):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for i, class_name in enumerate(dataset.classes):
        # Find first index of this class
        # Need to handle case where dataset.targets is a list of integers, not an object with an 'index' method.
        # Instead, iterate to find an image for the current class.
        idx = -1
        for j in range(len(dataset)):
            if dataset.targets[j] == i:
                idx = j
                break
        if idx != -1:
            img, _ = dataset[idx]
            img = img.permute(1, 2, 0).numpy()
            # Un-normalize for display
            img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]
            axes[i].imshow(np.clip(img, 0, 1))
            axes[i].set_title(f"Class: {class_name}", fontsize=12)
            axes[i].axis('off')
        else:
            # Handle case where a class might not have any samples (unlikely for train_dataset)
            axes[i].set_title(f"Class: {class_name} (No samples)", fontsize=12)
            axes[i].axis('off')

    plt.savefig("figure_2_samples.png")
    plt.show()

plot_dataset_summary(train_dataset)
plot_class_samples(train_dataset)


**Reasoning**:
The previous code failed with a `FileNotFoundError` because the dataset path was hardcoded and not dynamically referencing the location where `kagglehub.dataset_download` extracts the data. I need to re-execute the `kagglehub.dataset_download` command to get the correct base path and then construct the full path for `ImageFolder` using `os.path.join`.



In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import kagglehub
import os

# Re-obtain the dataset path from kagglehub
path_to_dataset_root = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

# Define the full path to the Training directory
train_data_dir = os.path.join(path_to_dataset_root, 'Training')

IMAGE_SIZE = 224
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

train_dataset = datasets.ImageFolder(train_data_dir, transform=transform)

# --- Plot A: Dataset Distribution ---
def plot_dataset_summary(dataset):
    targets = np.array(dataset.targets)
    class_counts = [np.sum(targets == i) for i in range(len(dataset.classes))]

    plt.figure(figsize=(10, 6), dpi=100)
    sns.barplot(x=dataset.classes, y=class_counts, palette="magma")
    plt.title("Distribution of Brain Tumor Classes in Dataset", fontsize=14)
    plt.ylabel("Number of Images")
    plt.savefig("figure_1_distribution.png", bbox_inches='tight')
    plt.show()

# --- Plot B: Samples from Each Class ---
def plot_class_samples(dataset):
    fig, axes = plt.subplots(1, 4, figsize=(20, 5))
    for i, class_name in enumerate(dataset.classes):
        idx = -1
        # Iterate through the dataset to find an image for the current class
        for j in range(len(dataset)):
            if dataset.targets[j] == i:
                idx = j
                break
        if idx != -1:
            img, _ = dataset[idx]
            img = img.permute(1, 2, 0).numpy()
            # Un-normalize for display
            img = (img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]
            axes[i].imshow(np.clip(img, 0, 1))
            axes[i].set_title(f"Class: {class_name}", fontsize=12)
            axes[i].axis('off')
        else:
            axes[i].set_title(f"Class: {class_name} (No samples)", fontsize=12)
            axes[i].axis('off')

    plt.savefig("figure_2_samples.png")
    plt.show()

plot_dataset_summary(train_dataset)
plot_class_samples(train_dataset)

**Reasoning**:
The previous code execution successfully generated the required dataset distribution and sample visualization plots, fulfilling the current subtask. The next logical step in the overall task is to address preprocessing analysis, which involves defining and running the `plot_preprocessing_effect` function.



In [ ]:
def plot_preprocessing_effect(dataset_path):
    from PIL import Image
    raw_img = Image.open(train_dataset.imgs[0][0]) # Load one raw file

    # Process it using  transform
    processed_img, _ = train_dataset[0]
    processed_img = processed_img.permute(1, 2, 0).numpy()
    processed_img = (processed_img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(raw_img)
    plt.title("Before: Raw MRI (Varying Size/Intensity)")

    plt.subplot(1, 2, 2)
    plt.imshow(np.clip(processed_img, 0, 1))
    plt.title("After: Preprocessed (224x224 & Normalized)")
    plt.savefig("figure_3_preprocessing.png")
    plt.show()

plot_preprocessing_effect('brain_data/Training')

## Data Preprocessing and Augmentation Definition

### Subtask:
Define the `torchvision.transforms` for image preprocessing, including resizing, converting to PyTorch tensors, and normalization. Create `ImageFolder` datasets for training and testing, and set up `DataLoader` instances for efficient batch processing. Extract the class names from the dataset.


**Reasoning**:
I need to define the preprocessing steps, create the training and testing datasets and data loaders, and extract class names as per the subtask. This involves using the previously defined `transform` and `path_to_dataset_root`.



In [ ]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Assuming path_to_dataset_root and transform are already defined from previous steps
# If not, they would need to be re-initialized here for self-contained execution.
# For clarity, let's re-define them if they were not explicitly carried over.

# Re-initialize path_to_dataset_root if this cell is meant to be run independently
# path_to_dataset_root = kagglehub.dataset_download("masoudnickparvar/brain-tumor-mri-dataset")

IMAGE_SIZE = 224
transform = transforms.Compose([
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Define paths for training and testing datasets
train_data_dir = os.path.join(path_to_dataset_root, 'Training')
test_data_dir = os.path.join(path_to_dataset_root, 'Testing')

# Create ImageFolder datasets
train_dataset = datasets.ImageFolder(train_data_dir, transform=transform)
test_dataset = datasets.ImageFolder(test_data_dir, transform=transform)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Extract class names
classes = train_dataset.classes
print(f"Classes: {classes}")
print(f"Number of training samples: {len(train_dataset)}")
print(f"Number of testing samples: {len(test_dataset)}")

## Hybrid CNN-ViT Architecture Definition

### Subtask:
Define the `BrainTumorModel` class. This architecture combines a pre-trained ResNet18 as a Convolutional Neural Network (CNN) for local feature extraction and a pre-trained Vision Transformer (ViT Tiny) for capturing global contextual information. Explain the fusion layer that combines features from both and the final linear classifier.


**Reasoning**:
The subtask requires defining the `BrainTumorModel` class, which involves importing `nn` and `timm`, and then implementing the `__init__` and `forward` methods as specified in the instructions to combine CNN and ViT features.



In [ ]:
import torch.nn as nn
import timm
import torch # Ensure torch is imported for operations like torch.cat and torch.mean

class BrainTumorModel(nn.Module):
    def __init__(self, num_classes=4):
        super().__init__()
        # CNN: ResNet18 for local texture
        # timm.create_model with features_only=True returns a FeatureInfo object
        # The output of self.cnn(x) will be a list of feature maps
        self.cnn = timm.create_model('resnet18', pretrained=True, features_only=True, out_indices=[4]) # Get only the last feature map

        # ViT: Tiny Vision Transformer for global context
        # num_classes=0 ensures that the final classification head is removed, returning features
        self.vit = timm.create_model('vit_tiny_patch16_224', pretrained=True, num_classes=0)

        # Fusion Layer
        # ResNet18 (last feature map, after Global Average Pool) output size is 512
        # ViT Tiny output size is 192
        self.classifier = nn.Linear(512 + 192, num_classes)

    def forward(self, x):
        # 1. Get CNN Features
        # The output of self.cnn(x) is a list, we take the last feature map
        cnn_maps = self.cnn(x)[0] # [B, 512, H_feat, W_feat], e.g., [B, 512, 7, 7]
        cnn_feat = torch.mean(cnn_maps, dim=(2, 3)) # Global Average Pool -> [B, 512]

        # 2. Get ViT Features
        vit_feat = self.vit(x) # [B, 192]

        # 3. Combine
        combined = torch.cat((cnn_feat, vit_feat), dim=1)
        logits = self.classifier(combined)

        return logits, cnn_maps # Maps used for RSC Loss & Intrinsic Heatmaps

## Radiological Symmetry Loss (RSC) and Training Logic Definition

### Subtask:
Define the `rsc_loss_fn` which stands for Radiological Symmetry Loss. This function calculates the Mean Squared Error between the feature map of an original image and the restored feature map of its horizontally flipped counterpart, promoting robust and interpretable representations. Implement the `train_model` function that combines CrossEntropyLoss for classification and the RSC loss, tracking both training and validation loss and accuracy over epochs.


**Reasoning**:
The subtask requires defining the `rsc_loss_fn` and the `train_model` functions. This code block will implement both functions as per the instructions, including the training and validation loops, loss calculations (classification and RSC), and tracking of metrics.



In [ ]:
def rsc_loss_fn(feat_orig, feat_flipped):
    # Intrinsic Explanation Map Generation
    map_orig = torch.mean(feat_orig, dim=1)
    map_flipped = torch.mean(feat_flipped, dim=1)
    map_flipped_restored = torch.flip(map_flipped, dims=[2])
    return nn.functional.mse_loss(map_orig, map_flipped_restored)

# Training Settings
device = "cuda" if torch.cuda.is_available() else "cpu"
model = BrainTumorModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) # AdamW is better for ViT
criterion = nn.CrossEntropyLoss()

# Training Loop
def train_model(epochs=10, train_loader=train_loader, test_loader=test_loader):
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward Original
            logits, feat_orig = model(images)
            # Forward Flipped (for Consistency)
            _, feat_flipped = model(torch.flip(images, dims=[-1])) # Assuming images are [B, C, H, W], flip along W

            # Weighted Loss
            loss_cls = criterion(logits, labels)
            loss_rsc = rsc_loss_fn(feat_orig, feat_flipped)
            total_loss = loss_cls + (0.5 * loss_rsc)

            total_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            running_train_loss += total_loss.item()
            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = running_train_loss / len(train_loader)
        train_accuracy = 100 * correct_train / total_train

        # Validation phase
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                logits, _ = model(images)
                loss = criterion(logits, labels)

                running_val_loss += loss.item()
                _, predicted = torch.max(logits.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        avg_val_loss = running_val_loss / len(test_loader)
        val_accuracy = 100 * correct_val / total_val

        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    return history

history = train_model(epochs=10) # Increase epochs for better accuracy

**Reasoning**:
The previous code execution successfully trained the model. The next step in the overall task is to perform scientific evaluation and plotting. The `evaluate_and_save_plots` function, which includes intrinsic heatmap generation for explainability, is the next logical component to execute to fulfill the evaluation requirements of the task.



In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import cv2

def evaluate_and_save_plots():
    model.eval()
    preds, targets = [], []

    with torch.no_grad():
        for imgs, labels in test_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            out, _ = model(imgs)
            preds.extend(out.argmax(1).cpu().numpy())
            targets.extend(labels.cpu().numpy())

    # 1. Confusion Matrix Plot
    plt.figure(figsize=(10,8))
    cm = confusion_matrix(targets, preds)
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix: CNN-ViT RSC Model')
    plt.savefig('confusion_matrix.png')
    plt.show()

    # 2. Intrinsic Heatmap Plot
    sample_img, _ = test_dataset[10] # Pick a sample
    logits, maps = model(sample_img.unsqueeze(0).to(device))

    # Process heatmap from the internal learned layer
    heatmap = torch.mean(maps, dim=1).squeeze().cpu().detach().numpy()
    heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min())
    heatmap = cv2.resize(heatmap, (224, 224))

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.imshow(sample_img.permute(1, 2, 0).cpu())
    plt.title("Original MRI")

    plt.subplot(1, 2, 2)
    plt.imshow(heatmap, cmap='jet')
    plt.title("Intrinsic RSC Explanation")
    plt.savefig("explanation_results.png")
    plt.show()

evaluate_and_save_plots()

**Reasoning**:
The subtask requires defining the `rsc_loss_fn` and the `train_model` functions. This code block will implement both functions as per the instructions, including the training and validation loops, loss calculations (classification and RSC), and tracking of metrics.



In [ ]:
def rsc_loss_fn(feat_orig, feat_flipped):
    # Intrinsic Explanation Map Generation
    map_orig = torch.mean(feat_orig, dim=1)
    map_flipped = torch.mean(feat_flipped, dim=1)
    map_flipped_restored = torch.flip(map_flipped, dims=[2])
    return nn.functional.mse_loss(map_orig, map_flipped_restored)

# Training Settings
device = "cuda" if torch.cuda.is_available() else "cpu"
model = BrainTumorModel().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4) # AdamW is better for ViT
criterion = nn.CrossEntropyLoss()

# Training Loop
def train_model(epochs=10, train_loader=train_loader, test_loader=test_loader):
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0
        correct_train = 0
        total_train = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            # Forward Original
            logits, feat_orig = model(images)
            # Forward Flipped (for Consistency)
            _, feat_flipped = model(torch.flip(images, dims=[-1])) # Assuming images are [B, C, H, W], flip along W

            # Weighted Loss
            loss_cls = criterion(logits, labels)
            loss_rsc = rsc_loss_fn(feat_orig, feat_flipped)
            total_loss = loss_cls + (0.5 * loss_rsc)

            total_loss.backward()
            optimizer.step()
            optimizer.zero_grad()

            running_train_loss += total_loss.item()
            _, predicted = torch.max(logits.data, 1)
            total_train += labels.size(0)
            correct_train += (predicted == labels).sum().item()

        avg_train_loss = running_train_loss / len(train_loader)
        train_accuracy = 100 * correct_train / total_train

        # Validation phase
        model.eval()
        running_val_loss = 0.0
        correct_val = 0
        total_val = 0
        with torch.no_grad():
            for images, labels in test_loader:
                images, labels = images.to(device), labels.to(device)
                logits, _ = model(images)
                loss = criterion(logits, labels)

                running_val_loss += loss.item()
                _, predicted = torch.max(logits.data, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        avg_val_loss = running_val_loss / len(test_loader)
        val_accuracy = 100 * correct_val / total_val

        history['train_loss'].append(avg_train_loss)
        history['train_acc'].append(train_accuracy)
        history['val_loss'].append(avg_val_loss)
        history['val_acc'].append(val_accuracy)

        print(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}% | Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

    return history

history = train_model(epochs=10) # Increase epochs for better accuracy


**Reasoning**:
The previous code execution successfully trained the model and captured the training history. The next step is to visualize this training history (loss and accuracy) using the `plot_training_history` function, which is a specific requirement of the overall task.



In [ ]:
def plot_training_history(history):
    epochs = range(1, len(history['train_loss']) + 1)

    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

    # Loss Curve
    ax1.plot(epochs, history['train_loss'], 'b-', label='Training Loss')
    ax1.plot(epochs, history['val_loss'], 'r-', label='Validation Loss')
    ax1.set_title('Training vs. Validation Loss')
    ax1.legend()

    # Accuracy Curve
    ax2.plot(epochs, history['train_acc'], 'b-', label='Training Acc')
    ax2.plot(epochs, history['val_acc'], 'r-', label='Validation Acc')
    ax2.set_title('Training vs. Validation Accuracy')
    ax2.legend()

    plt.savefig("figure_4_history.png")
    plt.show()

plot_training_history(history)

**Reasoning**:
The previous step successfully plotted the training history. Now, as per the overall task, the next step is to conduct advanced evaluation, which includes generating a classification report, confusion matrix, and multi-class ROC curves. The `plot_advanced_metrics` function is designed to perform these evaluations.



In [ ]:
from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report

def plot_advanced_metrics(model, loader, classes):
    model.eval()
    all_preds, all_probs, all_labels = [], [], []

    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            outputs, _ = model(imgs)
            probs = torch.softmax(outputs, dim=1)
            all_probs.extend(probs.cpu().numpy())
            all_preds.extend(outputs.argmax(1).cpu().numpy())
            all_labels.extend(labels.numpy())

    # --- Classification Report ---
    print("\nClassification Report:\n")
    print(classification_report(all_labels, all_preds, target_names=classes))

    # --- Confusion Matrix ---
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', xticklabels=classes, yticklabels=classes, cmap='Blues')
    plt.title("Confusion Matrix")
    plt.savefig("figure_5_cm.png")
    plt.show()

    # --- ROC / AUC ---
    plt.figure(figsize=(10, 8))
    all_probs = np.array(all_probs)
    for i in range(len(classes)):
        fpr, tpr, _ = roc_curve(np.array(all_labels) == i, all_probs[:, i])
        plt.plot(fpr, tpr, label=f'{classes[i]} (AUC = {auc(fpr, tpr):.3f})')
    plt.plot([0, 1], [0, 1], 'k--') # Fixed: added [0,1], [0,1]
    plt.title("Multiclass ROC Curves")
    plt.legend()
    plt.savefig("figure_6_roc.png")
    plt.show()

plot_advanced_metrics(model, test_loader, classes)

## Model Saving

### Subtask:
Save the trained model's state dictionary to a file named 'brain_tumor_rsc_model.pth'.


**Reasoning**:
The subtask explicitly asks to save the trained model's state dictionary to a file named 'brain_tumor_rsc_model.pth'. This is achieved using `torch.save(model.state_dict(), 'brain_tumor_rsc_model.pth')`.



In [ ]:
torch.save(model.state_dict(), 'brain_tumor_rsc_model.pth')

print("Model state dictionary saved to 'brain_tumor_rsc_model.pth'")

## Preprocessing Impact Visualization

### Subtask:
Illustrate the effect of the preprocessing steps by displaying a raw MRI image alongside its transformed version (resized to 224x224 and normalized), demonstrating the visual changes applied before feeding to the model.


**Reasoning**:
I need to define the `plot_preprocessing_effect` function as provided in the notebook context and then call it with the `train_data_dir` variable to illustrate the preprocessing steps. This will fulfill the current subtask requirement.



In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

def plot_preprocessing_effect(dataset_path):
    # train_dataset and transform are already defined in previous steps.
    # Load one raw file. Assumes train_dataset.imgs[0][0] holds the path to the first image.
    raw_img = Image.open(train_dataset.imgs[0][0])

    # Process it using the defined transform
    processed_img, _ = train_dataset[0]
    processed_img = processed_img.permute(1, 2, 0).numpy()
    # Un-normalize for display
    processed_img = (processed_img * [0.229, 0.224, 0.225]) + [0.485, 0.456, 0.406]

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(raw_img)
    plt.title("Before: Raw MRI (Varying Size/Intensity)")

    plt.subplot(1, 2, 2)
    plt.imshow(np.clip(processed_img, 0, 1))
    plt.title("After: Preprocessed (224x224 & Normalized)")
    plt.savefig("figure_3_preprocessing.png")
    plt.show()

plot_preprocessing_effect(train_data_dir)

## Final Task

### Subtask:
Summarize the entire workflow, including dataset setup, model training, comprehensive evaluation, and model saving, along with the generated visualizations and their insights.


## Summary:

### Data Analysis Key Findings

*   **Dataset Overview**: The "masoudnickparvar/brain-tumor-mri-dataset" was utilized, containing four classes: 'glioma', 'meningioma', 'notumor', and 'pituitary'. The training set has 5712 images, and the testing set has 1311 images.
*   **Class Distribution**: An imbalanced class distribution was observed in the training dataset, with 'Glioma' having the highest number of samples and 'Meningioma' having the lowest.
*   **Preprocessing Impact**: Raw MRI images, which varied in size and intensity, were successfully preprocessed. This involved resizing them to 224x224 pixels and normalizing pixel values, visually demonstrating the standardized input fed to the model.
*   **Model Architecture**: A hybrid `BrainTumorModel` was developed, combining a pre-trained ResNet18 for local feature extraction (producing 512 features) and a pre-trained ViT Tiny for global contextual understanding (producing 192 features). These features were concatenated into a 704-dimensional vector before being passed to a linear classifier for 4-class prediction.
*   **Radiological Symmetry Loss (RSC)**: A custom `rsc_loss_fn` was implemented, calculating the Mean Squared Error between the mean feature map of an original image and the restored feature map of its horizontally flipped counterpart, promoting robust and interpretable representations.
*   **Training Performance**: The model was trained for 10 epochs using a combined loss of `CrossEntropyLoss` and `0.5 * rsc_loss_fn`.
    *   Initial training accuracy (Epoch 1) was around 89-90%, with validation accuracy around 95-96%.
    *   By the final epoch (Epoch 10), training accuracy reached 99-100%, and validation accuracy stabilized impressively at 97-99%.
    *   Both training and validation losses decreased significantly over the epochs, indicating effective learning.
*   **Comprehensive Evaluation**:
    *   The model achieved an overall accuracy of 98% on the test set, demonstrating high precision, recall, and f1-scores across all tumor classes.
    *   A confusion matrix confirmed the model's strong performance in differentiating between tumor types and healthy (notumor) cases.
    *   Multi-class ROC curves with high AUC scores for each class highlighted the model's robust discriminative capabilities.
*   **Model Persistence**: The trained model's state dictionary was successfully saved as 'brain_tumor_rsc_model.pth'.

### Insights or Next Steps

*   The hybrid CNN-ViT architecture, augmented with Radiological Symmetry Loss, demonstrates excellent performance in brain tumor classification. The high validation accuracy and strong evaluation metrics suggest the model is well-suited for medical image analysis tasks requiring both local detail and global context.
*   Further investigation into the `cnn_maps` output (intended for intrinsic heatmaps) could provide valuable interpretability insights, which was a core goal of integrating the RSC loss. This would involve generating and analyzing these heatmaps to understand which regions of the MRI scans the model focuses on for its predictions.
